In [1]:
# convert decimal lat / lon in WGS83 EPSG: 4326 to NAD83(NRS2007) UTM meters, Zone 17N EPSG:3724
# to get this to work, I had to define GDAL_DATA in my Win environment path

# csherwood@usgs.gov
# 4 November 2016

Disclaimer

Unless otherwise noted, the software and content on this site is
in the public domain because it contains materials developed by
the United States Geological Survey, an agency of the United States
Department of Interior. For more information, see the official USGS
copyright policy at:

http://www.usgs.gov/visual-id/credit_usgs.html#copyright

This software and content is distributed on an "AS IS" BASIS, WITHOUT
WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. Any
dependent libraries included here are distributed under open source
(or open source-like) licenses/agreements. Appropriate license agreements
are included with each library.

In [2]:
import numpy as np

# GDAL Python library
from osgeo import osr
# EXIF Reader
import exifread
# Image library
#from PIL import Image

from IPython.display import Image
Image("vdatum_screen_grab.jpg")

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

src_crs = 'EPSG:4326'
dst_crs = 'EPSG:3724'

# source coordinate system
srs_cs = osr.SpatialReference()
srs_cs.SetFromUserInput(src_crs)

# destination coordinate system
dst_cs = osr.SpatialReference()
dst_cs.SetFromUserInput(dst_crs)

# osr image transformation object
transform = osr.CoordinateTransformation(srs_cs, dst_cs)

# print coordinate system information
print " >> SOURCE COORDINATE REFERENCE SYSTEM: "
print srs_cs

print " >> DESTINATION COORDINATE REFERENCE SYSTEM: "
print dst_cs


 >> SOURCE COORDINATE REFERENCE SYSTEM: 
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
 >> DESTINATION COORDINATE REFERENCE SYSTEM: 
PROJCS["NAD83(NSRS2007) / UTM zone 17N",
    GEOGCS["NAD83(NSRS2007)",
        DATUM["NAD83_National_Spatial_Reference_System_2007",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6759"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4759"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["cen

In [3]:
    # I checked this transform using VDATUM for the Point 1 south below, with source:
    # Source: "WGS84(transit) - use NAD83 (see NGS's HTDP)"
    # Target: "NAVD88" using GEOID12B
    # northing and easting transforms were reproduced exactly
    # for Point 1 south, height was 1.0326 m NADV88, with vertical uncertainty of 7.6158 cm
    # deltaz = 28.7909
    # for Point 9 north, deltaz - 28.7911
    # so, using:
    deltaz = 28.7910

In [4]:
fnin = "D:\\crs\\proj\\2017_Matthew\\maitane\\2016-10-30_control_points.csv"
fnout = "D:\\crs\\proj\\2017_Matthew\\maitane\\2016-10-30_GCP_UTM.csv"

header_out = "Label, Lat (WGS84), Lon (WGS84), Alt. (m), Easting (m NAD83 UTM17N), Northing (m NAD83 UTM17N), Elev. (m NADV88)"
f=open(fnin,'r')
fo = open(fnout, 'w')
header = f.readline()
print header
fo.write(header_out+"\n")
for line in f:
    line = line.strip()
    col = line.split(',')
    name = col[0]
    lat = float(col[1])+float(col[2])/60.+float(col[3])/3600.
    lon = -(float(col[4])+float(col[5])/60.+float(col[6])/3600.)
    ze = float(col[7])
    e,n,zt = transform.TransformPoint(lon, lat, ze)
    znavd88 = zt+deltaz
    #print(name, lat, lon, ze, e, n, znavd88)
    print("{0:s}, {1:.6f}, {2:.6f}, {3:.4f}, {4:.4f}, {5:.4f}, {6:.4f}".format(name,lat,lon,ze,e,n,znavd88))
    fo.write("{0:s}, {1:.6f}, {2:.6f}, {3:.4f}, {4:.4f}, {5:.4f}, {6:.4f}\n".format(name,lat,lon,ze,e,n,znavd88))
f.close
fo.close

Label,Lat (degrees),Lat (minutes),Lat (seconds),Lon (degrees),Lon (minutes),Lon (seconds),Z (Height above the WGS84 ellipsoide),Lat (WGS84),Lon(WGS84)

Point 1 south, 29.683666, -81.218714, -27.7583, 478838.9754, 3283753.8587, 1.0327
Point2 south, 29.683818, -81.218888, -28.6757, 478822.1098, 3283770.7518, 0.1153
Point3 south, 29.683666, -81.218937, -28.7528, 478817.3594, 3283753.8340, 0.0382
Point4 south, 29.683769, -81.218988, -28.5370, 478812.4735, 3283765.2579, 0.2540
Point5 south, 29.683624, -81.218852, -28.1598, 478825.6177, 3283749.2133, 0.6312
Point6 south, 29.683519, -81.218881, -27.6961, 478822.7810, 3283737.5846, 1.0949
Point7 south, 29.683355, -81.218757, -27.5410, 478834.6762, 3283719.3903, 1.2500
Point8 south, 29.683599, -81.218536, -28.6292, 478856.1846, 3283746.3694, 0.1618
Point 1 north, 29.685358, -81.219510, -27.7330, 478762.2574, 3283941.4009, 1.0580
Point 2 north, 29.685358, -81.219427, -27.5740, 478770.2724, 3283941.4085, 1.2170
Point 3 north, 29.685595, -81.21942

<function close>